# 📘 RoBERTa Model Training for Cloud Threat Classification

This notebook demonstrates how to train a `roberta-base` model to classify cloud threats into multiple security attributes.

In [ ]:
!pip install transformers datasets torch scikit-learn

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset

In [ ]:
df = pd.read_csv("../data/cloud_security_challenges.csv")
X = df["Challenge"]
y = df.drop(columns=["Challenge"]).values.tolist()
labels = df.columns[1:].tolist()

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
class ThreatDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx], padding='max_length', truncation=True,
            max_length=self.max_len, return_tensors="pt"
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return inputs

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
train_dataset = ThreatDataset(X_train.tolist(), y_train, tokenizer)
test_dataset = ThreatDataset(X_test.tolist(), y_test, tokenizer)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=len(labels), problem_type="multi_label_classification")

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
trainer.train()

In [ ]:
predictions = trainer.predict(test_dataset).predictions
y_pred = (predictions > 0.5).astype(int)
print(classification_report(y_test, y_pred, target_names=labels))